In [3]:
import pandas as pd          # Manejo y análisis de datos en tablas (DataFrames)
from IPython.display import display  # Muestra objetos de forma clara en notebooks
import matplotlib.pyplot as plt      # Crear gráficos y visualizaciones
import seaborn as sns        # Gráficos estadísticos atractivos (basado en matplotlib)
from matplotlib.lines import Line2D  # Crear o personalizar líneas en gráficos



In [4]:
df_listado = pd.read_excel("../Data/musicales_listado.xlsx")
df_listado.head(20)

,obra,productora,anio_inicio,anio_fin,teatro,ciudad principal,gira,fuente_ url,activa,duracion
0,Annie,Theatre Properties,2010,2011,Nuevo Alcalá,Madrid,No,https://www.carteleramusicales.es/,False,120
1,Avenue Q,SMedia,2010,2011,Nuevo Apolo,Madrid,No,https://www.carteleramusicales.es/,False,135
2,Hair,The William Morris Agency Endeavor Entertainme...,2010,2011,Apolo,Barcelona,No,https://www.carteleramusicales.es/,False,150
3,Los miserables,Stage Entertainment,2010,2012,Lope de Vega,Madrid,Si,https://www.stage.es/musicales/los-miserables,False,170
4,Pegados,Ferran González & Alícia Serrat,2010,2013,Almería Teatre,Barcelona,Si,Pegados (2010) - Cartelera Musicales,False,80
5,Forever Young,Tricicle,2011,2022,Edp Gran Via,Madrid,Si,Forever Young - Compañía Teatral Tricicle,False,100
6,Grease,Drive Entertainment,2011,2014,Las Arenas,Barcelona,Si,Grease (2011) - Cartelera Musicales,False,150
7,Mas de 100 mentiras,Drive Entertainment,2011,2013,Rialto,Madrid,Si,Más de 100 mentiras (2011) - Cartelera Musicales,False,170
8,El último jinete,Arabian Horses Production LTD,2012,2013,Teatros del Canal,Madrid,No,https://www.carteleramusicales.es/el-ultimo-ji...,False,130
9,Follies,Mario Gas,2012,2013,Español,Madrid,No,https://www.carteleramusicales.es/follies-2012,False,160


In [5]:
df_listado.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   obra               33 non-null     object
 1   productora         33 non-null     object
 2   anio_inicio        33 non-null     int64 
 3   anio_fin           33 non-null     int64 
 4   teatro             33 non-null     object
 5   ciudad principal   33 non-null     object
 6   gira               33 non-null     object
 7   fuente_ url        33 non-null     object
 8   activa             33 non-null     object
 9   duracion           33 non-null     int64 
dtypes: int64(3), object(7)
memory usage: 2.7+ KB


In [6]:

# 1. Cargar datasets
df_limpio = pd.read_csv("../Data/musicales_limpio.csv")
df_listado = pd.read_excel("../Data/musicales_listado.xlsx")

# 2. Ver columnas
print(df_limpio.columns)
print(df_listado.columns)



Index(['obra', 'productora', 'anio_inicio', 'anio_fin', 'teatro',
       'ciudad_principal', 'gira', 'fuente_url', 'activa', 'duracion'],
      dtype='object')
Index(['obra ', 'productora', 'anio_inicio', 'anio_fin', 'teatro',
       'ciudad principal ', 'gira', 'fuente_ url', 'activa', 'duracion'],
      dtype='object')


In [7]:
# 1) LIMPIAR NOMBRES DE COLUMNAS (por si hay espacios)
df_limpio.columns  = df_limpio.columns.str.strip()
df_listado.columns = df_listado.columns.str.strip()

# 2) ASEGURAR QUE 'obra' EXISTE (y limpiar texto dentro de las celdas)
#    Esto evita espacios raros dentro del título (no solo en el nombre de la columna)
df_limpio["obra"]  = df_limpio["obra"].astype(str).str.strip()
df_listado["obra"] = df_listado["obra"].astype(str).str.strip()

# 3) QUITAR DURACION DE AMBOS ANTES DE CONCATENAR (para que NO venga del CSV)
df_limpio  = df_limpio.drop(columns=["duracion"], errors="ignore")
df_listado = df_listado.drop(columns=["duracion"], errors="ignore")

# 4) CONCATENAR USANDO SOLO COLUMNAS COMUNES (pero DESPUÉS de asegurar 'obra')
columnas_comunes = df_limpio.columns.intersection(df_listado.columns)
df_maestro = pd.concat(
    [df_limpio[columnas_comunes], df_listado[columnas_comunes]],
    ignore_index=True
)

# 5) RECARGAR EXCEL COMPLETO PARA TRAER DURACIONES
df_listado_full = pd.read_excel("../Data/musicales_listado.xlsx")
df_listado_full.columns = df_listado_full.columns.str.strip()

# 6) CREAR CLAVES PARA MERGE (robusto)
df_maestro["obra_key"] = df_maestro["obra"].astype(str).str.strip().str.lower()
df_listado_full["obra"] = df_listado_full["obra"].astype(str).str.strip()
df_listado_full["obra_key"] = df_listado_full["obra"].str.strip().str.lower()

# 7) TRAER DURACION SOLO DEL EXCEL (left join)
#    Resultado: duracion solo para las obras que estén en el Excel; el resto NaN
df_maestro = df_maestro.merge(
    df_listado_full[["obra_key", "duracion"]],
    on="obra_key",
    how="left"
)

# 8) LIMPIEZA FINAL
df_maestro = df_maestro.drop(columns=["obra_key"])


In [8]:
print(df_limpio.columns)
print(df_listado.columns)


Index(['obra', 'productora', 'anio_inicio', 'anio_fin', 'teatro',
       'ciudad_principal', 'gira', 'fuente_url', 'activa'],
      dtype='object')
Index(['obra', 'productora', 'anio_inicio', 'anio_fin', 'teatro',
       'ciudad principal', 'gira', 'fuente_ url', 'activa'],
      dtype='object')


In [9]:
df_maestro.head(30)


,obra,productora,anio_inicio,anio_fin,teatro,gira,activa,duracion
0,"101 Dálmatas, el musical",Teatropolis (Gran Teatro CaixaBank Príncipe Pío),2023,NaN,Gran Teatro CaixaBank Príncipe Pío,No,True,NaN
1,Aladdín,Stage Entertainment,2023,2025.0,Teatro Coliseum,No,False,NaN
2,Anastasia,Stage Entertainment,2018,2020.0,Teatro Coliseum,No,False,NaN
3,Avenue Q,Teatropolis (Gran Teatro CaixaBank Príncipe Pío),2024,NaN,Gran Teatro CaixaBank Príncipe Pío,No,True,135.0
4,Billy Elliot,SOM Produce,2017,2020.0,Nuevo Teatro Alcalá,Sí,False,NaN
5,Cabaret,Let's Go Company,2025,NaN,Albéniz,No,True,NaN
6,Cenicienta,Stage Entertainment,2025,NaN,Coliseum,No,True,NaN
7,Charlie y la fábrica de chocolate,Let's Go Company,2021,2022.0,Espacio Ibercaja Delicias,Sí,False,NaN
8,Chicago,SOM Produce,2023,2025.0,Teatro Apolo,Sí,False,NaN
9,Dirty Dancing,Let's Go Company,2018,2023.0,Teatro Nuevo Alcalá,Sí,False,NaN


In [10]:
df_maestro.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   obra         77 non-null     object 
 1   productora   77 non-null     object 
 2   anio_inicio  77 non-null     int64  
 3   anio_fin     64 non-null     float64
 4   teatro       77 non-null     object 
 5   gira         77 non-null     object 
 6   activa       77 non-null     object 
 7   duracion     41 non-null     float64
dtypes: float64(2), int64(1), object(5)
memory usage: 4.9+ KB


In [11]:
df_maestro.describe


<bound method NDFrame.describe of                         obra  \
0   101 Dálmatas, el musical   
1                    Aladdín   
2                  Anastasia   
3                   Avenue Q   
4               Billy Elliot   
..                       ...   
72            Come from away   
73      El día de la marmota   
74                     Gypsy   
75   Tocando nuestra canción   
76      La movida,el musical   

                                          productora  anio_inicio  anio_fin  \
0   Teatropolis (Gran Teatro CaixaBank Príncipe Pío)         2023       NaN   
1                                Stage Entertainment         2023    2025.0   
2                                Stage Entertainment         2018    2020.0   
3   Teatropolis (Gran Teatro CaixaBank Príncipe Pío)         2024       NaN   
4                                        SOM Produce         2017    2020.0   
..                                               ...          ...       ...   
72                          

In [12]:
# 1) Cargar Excel de obras sin duración
df_sin_duracion = pd.read_excel("../Data/obras_sin_duracion.xlsx")

# 2) Normalizar nombres de columnas
df_sin_duracion.columns = df_sin_duracion.columns.str.strip()

# 3) Crear clave de cruce
df_maestro["obra_key"] = df_maestro["obra"].str.strip().str.lower()
df_sin_duracion["obra_key"] = df_sin_duracion["obra"].str.strip().str.lower()

# 4) Merge para traer la nueva duración
df_maestro = df_maestro.merge(
    df_sin_duracion[["obra_key", "duracion"]],
    on="obra_key",
    how="left",
    suffixes=("", "_nueva")
)

# 5) Rellenar SOLO los NaN con la duración nueva
df_maestro["duracion"] = df_maestro["duracion"].fillna(df_maestro["duracion_nueva"])

# 6) Limpieza final
df_maestro = df_maestro.drop(columns=["duracion_nueva", "obra_key"])


FileNotFoundError: [Errno 2] No such file or directory: '../Data/obras_sin_duracion.xlsx'

In [ ]:
df_maestro[df_maestro["duracion"].notna()].shape


(77, 8)

In [ ]:
df_maestro["duracion"].notna().sum()
df_maestro["duracion"].isna().sum()


np.int64(0)

In [ ]:
df_maestro.shape



(77, 8)

In [ ]:
df_maestro.columns

Index(['obra', 'productora', 'anio_inicio', 'anio_fin', 'teatro', 'gira',
       'activa', 'duracion'],
      dtype='object')

In [ ]:
df_maestro.head(30)

,obra,productora,anio_inicio,anio_fin,teatro,gira,activa,duracion
0,"101 Dálmatas, el musical",Teatropolis (Gran Teatro CaixaBank Príncipe Pío),2023,NaN,Gran Teatro CaixaBank Príncipe Pío,No,True,90.0
1,Aladdín,Stage Entertainment,2023,2025.0,Teatro Coliseum,No,False,145.0
2,Anastasia,Stage Entertainment,2018,2020.0,Teatro Coliseum,No,False,95.0
3,Avenue Q,Teatropolis (Gran Teatro CaixaBank Príncipe Pío),2024,NaN,Gran Teatro CaixaBank Príncipe Pío,No,True,135.0
4,Billy Elliot,SOM Produce,2017,2020.0,Nuevo Teatro Alcalá,Sí,False,150.0
5,Cabaret,Let's Go Company,2025,NaN,Albéniz,No,True,150.0
6,Cenicienta,Stage Entertainment,2025,NaN,Coliseum,No,True,145.0
7,Charlie y la fábrica de chocolate,Let's Go Company,2021,2022.0,Espacio Ibercaja Delicias,Sí,False,150.0
8,Chicago,SOM Produce,2023,2025.0,Teatro Apolo,Sí,False,140.0
9,Dirty Dancing,Let's Go Company,2018,2023.0,Teatro Nuevo Alcalá,Sí,False,145.0


In [ ]:
df_maestro.loc[
    df_maestro["obra"] == "Golfus de Roma",
    ["obra", "productora", "anio_inicio", "teatro", "duracion"]
]


,obra,productora,anio_inicio,teatro,duracion
58,Golfus de Roma,Euroscena,2015,Veranos de la Villa,90.0
59,Golfus de Roma,Euroscena,2015,Veranos de la Villa,155.0
67,Golfus de Roma,Pentación Espectáculos\n,2022,La latina,90.0
68,Golfus de Roma,Pentación Espectáculos\n,2022,La latina,155.0


In [ ]:
# ================================
# FIX FINAL: "Golfus de Roma"
# ================================

# 1) Corregir duración 90 -> 95
df_maestro.loc[
    (df_maestro["obra"] == "Golfus de Roma") &
    (df_maestro["duracion"] == 90),
    "duracion"
] = 95

# 2) Quedarse con 1 fila por versión (obra + productora + año)
#    Esto elimina SOLO la duplicada interna de 2015 y la duplicada interna de 2022.
df_maestro = (
    df_maestro
    .sort_values(["obra", "productora", "anio_inicio", "duracion"], ascending=[True, True, True, False])
    .drop_duplicates(subset=["obra", "productora", "anio_inicio"], keep="first")
    .reset_index(drop=True)
)

# 3) Verificación: deben quedar EXACTAMENTE 2 filas
out = df_maestro.loc[
    df_maestro["obra"] == "Golfus de Roma",
    ["obra", "productora", "anio_inicio", "teatro", "duracion"]
].sort_values(["anio_inicio", "productora"])

display(out)
print("Filas 'Golfus de Roma':", out.shape[0])


,obra,productora,anio_inicio,teatro,duracion
30,Golfus de Roma,Euroscena,2015,Veranos de la Villa,155.0
31,Golfus de Roma,Pentación Espectáculos\n,2022,La latina,155.0


Filas 'Golfus de Roma': 2


In [ ]:
df_maestro.loc[
    (df_maestro["obra"] == "Golfus de Roma") &
    (df_maestro["duracion"] == 155),
    "duracion"
] = 93


In [ ]:
df_maestro.loc[
    df_maestro["obra"] == "Golfus de Roma",
    ["obra", "productora", "anio_inicio", "duracion"]
]


,obra,productora,anio_inicio,duracion
30,Golfus de Roma,Euroscena,2015,93.0
31,Golfus de Roma,Pentación Espectáculos\n,2022,93.0


In [ ]:
# Normalización segura (no convierte desconocidos en NaN)
map_si_no = {
    "sí": "Sí", "si": "Sí", "s": "Sí", "yes": "Sí", "y": "Sí", "true": "Sí", "1": "Sí",
    "no": "No", "n": "No", "false": "No", "0": "No"
}

for col in ["gira", "activa"]:
    if col in df_maestro.columns:
        df_maestro[col] = (
            df_maestro[col]
            .astype(str)
            .str.strip()
            .str.lower()
            .replace(map_si_no)   # CLAVE: replace, no map
        )
        # Si había NaN reales, los devuelve a NaN (opcional, pero limpio)
        df_maestro.loc[df_maestro[col].isin(["nan", "none", ""]), col] = pd.NA


In [ ]:
df_maestro[["gira","activa"]].value_counts(dropna=False)


gira  activa
Sí    No        40
No    No        22
      Sí         9
Sí    Sí         4
Name: count, dtype: int64

In [ ]:
df_maestro["activa"] = (
    df_maestro["activa"]
    .astype(str)
    .str.strip()
    .str.lower()
    .replace({
        "sí": True, "si": True, "s": True, "true": True, "1": True,
        "no": False, "n": False, "false": False, "0": False
    })
)


C:\Users\elaru\AppData\Local\Temp\ipykernel_14504\3097304972.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({


In [ ]:
df_maestro["activa"].dtype
df_maestro["activa"].value_counts(dropna=False)


activa
False    62
True     13
Name: count, dtype: int64

In [ ]:
df_maestro.head(20)


,obra,productora,anio_inicio,anio_fin,teatro,gira,activa,duracion
0,"101 Dálmatas, el musical",Teatropolis (Gran Teatro CaixaBank Príncipe Pío),2023,NaN,Gran Teatro CaixaBank Príncipe Pío,No,Sí,90.0
1,Aladdín,Stage Entertainment,2023,2025.0,Teatro Coliseum,No,No,145.0
2,Anastasia,Stage Entertainment,2018,2020.0,Teatro Coliseum,No,No,95.0
3,Annie,Theatre Properties,2010,2011.0,Nuevo Alcalá,No,No,120.0
4,Avenue Q,SMedia,2010,2011.0,Nuevo Apolo,No,No,135.0
5,Avenue Q,Teatropolis (Gran Teatro CaixaBank Príncipe Pío),2024,NaN,Gran Teatro CaixaBank Príncipe Pío,No,Sí,135.0
6,Billy Elliot,SOM Produce,2017,2020.0,Nuevo Teatro Alcalá,Sí,No,150.0
7,Cabaret,Let's Go Company,2025,NaN,Albéniz,No,Sí,150.0
8,Cantando bajo la lluvia,Nostromo live,2021,2023.0,Tívoli,Sí,No,150.0
9,Cenicienta,Stage Entertainment,2025,NaN,Coliseum,No,Sí,145.0


In [ ]:
# 9) GUARDAR
df_maestro.to_csv("../Data/maestro_musicales.csv", index=False, encoding="utf-8")